In [36]:
import sys, re

# https://github.com/fzinfz/meter-dlt645
sys.path.append("/data/powermeter_dlt645/meter-dlt645")

from test_dlt645 import *
import test_dlt645

%load_ext autoreload
%autoreload 2

from power_meter import *
from power_meter_write import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
!ls /dev/ttyUSB*

/dev/ttyUSB0  /dev/ttyUSB1


In [3]:
verbose=0

serial_port = '/dev/ttyUSB0'
timeout_count = 10
wait_for_read = 0.5

chn=dlt645.Channel(port_id = serial_port, tmo_cnt = timeout_count, wait_for_read = wait_for_read)
chn.open()

True

# auto detect address

In [4]:
read_meter_address(chn, verbose)
addr = chn.rx_addr
print(addr)


--- Read meter address with broadcase mode ---
Meter address: 001522454104
[0, 21, 34, 69, 65, 4]


In [7]:
# algorithm for converting back to human readable
''.join([ '{0:0>2x}'.format(s) for s in addr ])

'001522454104'

# manually set address

In [48]:
addr = [ int(s,16) for s in re.findall('..', '000102866265') ]

# Read meter

In [57]:
read_meter(chn, addr)

{'电能-组合有功总-当前': (19935.28, 'kWh'),
 '电能-组合有功总-上结算日': (19928.71, 'kWh'),
 'A相电压': (233.2, 'V'),
 'A相电流': (0.716, 'A'),
 '功率-瞬时总有功': ('158.80', 'W'),
 '功率-一分钟有功总平均': ('158.70', 'W'),
 '功率因数-总': (0.944, ''),
 '表内温度': (0.2, '°C'),
 '内部电池电压': (3.65, 'V'),
 '内部电池工作时间': ('2.58', '年'),
 '表号': ('000102866265', ''),
 '每月第1结算日': (1.0, '日.时'),
 '每月第2结算日': (99.99, '日.时'),
 '每月第3结算日': (99.99, '日.时'),
 '功率-A相': ('166.97', 'W'),
 '电能:本周期': ('6.57', 'kWh'),
 '日期时间': ('20201101 12:41:42', '')}

In [10]:
read_last_outage_timestamp(chn, addr, 1, verbose)

for m in range(0,2):
    for n in [0,2,4]:
        read_energy(chn, addr, m, n, verbose)


--- Read last outage timestamp of N = 1 ---
Last outage timestamp: ff-1f-35 32:06:30 ----> ff-1b-35 3f:7f:22 

--- Read energy (Current Month) ---
Total: 1.48 kWh

--- Read energy (Current Month) ---
Peak: 0.00 kWh

--- Read energy (Current Month) ---
Trough: 1.48 kWh

--- Read energy (Last 1 Month) ---
Total: 0.00 kWh

--- Read energy (Last 1 Month) ---
Peak: 0.00 kWh

--- Read energy (Last 1 Month) ---
Trough: 0.00 kWh


# change Date & Time
https://github.com/glx-technologies/meter-dlt645  
运行前开前挡板按编程键

    rx_ctrl    : 94  => success
    rx_ctrl    : d4  => fail

In [46]:
!date

2020年 11月 01日 星期日 12:37:15 CST


In [44]:
change_date(chn, addr)

tx_frame   : fe fe 68 88 83 06 12 00 00 68 14 10 34 34 33 37 37 44 44 44 33 33 33 33 33 34 44 53 b6 16 
tx_addr    : 00 00 12 06 83 88 
tx_ctrl    : 14 
tx_payload : 01 01 00 04 04 11 11 11 00 00 00 00 00 01 11 20 
wait...

rx_frame   : 68 88 83 06 12 00 00 68 94 00 87 16 
rx_addr    : 00 00 12 06 83 88 
rx_ctrl    : 94 
rx_payload : 



In [50]:
change_time(chn, addr)

tx_frame   : fe fe 68 65 62 86 02 01 00 68 14 0f 35 34 33 37 37 44 44 44 33 33 33 33 89 73 45 26 16 
tx_addr    : 00 01 02 86 62 65 
tx_ctrl    : 14 
tx_payload : 02 01 00 04 04 11 11 11 00 00 00 00 56 40 12 
wait...

rx_frame   : 68 65 62 86 02 01 00 68 d4 01 37 2c 16 
rx_addr    : 00 01 02 86 62 65 
rx_ctrl    : d4 
rx_payload : 04 



# close channel

In [7]:
chn.close()